# Examples with run file and C++

This notebooks shows how to **write and execute run files** and **call C++ functions** using the interface from **consav**.

# Setup

In [1]:
%matplotlib inline

# reload module each time cell is run
%load_ext autoreload
%autoreload 2

# load the BufferStockModel module
from BufferStockModel import BufferStockModelClass

# use 8 threads in numba
from consav import runtools
runtools.write_numba_config(disable=0,threads=8)

# Run files

You can create a run file (here **run.py**) calling a specific method (here **test()**), and then run it in a system command.

In [2]:
model = BufferStockModelClass(solmethod='egm')
model.write_run_file(filename='run.py',method='test')
!python run.py

solution time: 0.5 secs
checksum: 3.6669553712300376


This is e.g. beneficial when testing how much speed-up numba provides:

In [3]:
from consav import runtools
model.write_run_file(filename='run.py',solmethod='egm',method='test',Np=1500,Nm=1500,Na=1500)

for threads in [8,4,2,1]:
    runtools.write_numba_config(threads=threads,threading_layer='omp') # threading_layer='tbb'
    print(f'threads = {threads}')
    !python run.py
    print('')

threads = 8
solution time: 2.7 secs
checksum: 3.401607450896838

threads = 4
solution time: 5.0 secs
checksum: 3.401607450896838

threads = 2
solution time: 9.0 secs
checksum: 3.401607450896838

threads = 1
solution time: 17.4 secs
checksum: 3.401607450896838



## C++

**Note:** The code below requires installing one of the following two compilers:

* **vs**: Free *Microsoft Visual Studio 2017 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Costly *Intel Parallel Studio 2018 Composer Edition* ([link](https://software.intel.com/en-us/parallel-studio-xe))

**Troubleshooting:** The _installation paths_ might need to be adjusted. See arguements the arguments to _cpptools.compile()_ function.

*Additional details are provided in the notebook **Calling C++** in the **Numba and C++** folder.*

### Overview

The **ConsumptionSavingModel** class also have an interface to C++. 

All C++ files should be in **cppfuncs/**. The main file should:

1. Include an **export macro**

  ```
  #define EXPORT extern "C" __declspec(dllexport)
  ```


2. Include the following **struct definitions**:

  ```
  #include "par_struct.cpp"
  #include "sol_struct.cpp"
  #include "sim_struct.cpp"  
  ```
  
  
3. Define **gateway functions** as:

  ```
  EXPORT void myfunction(par_struct *par, sol_struct *sol, sim_struct *sim)
  ```


4. If using visual studio the following function should be included:

  ```
  EXPORT void setup_omp(){
      SetEnvironmentVariable("OMP_WAIT_POLICY", "passive"); 
  }
  ```   

### Linking

The most import methods are:

1. **setup_cpp(compiler)**: Chooses the compiler (_vs_ or _intel_) and writes *par\_struct.cpp*, *sol\_struct.cpp*, *sim\_strunct.cpp* in cppfuncs/.
* **link_cpp(FILE,FUNCS)**: Link to C++ file **cppfuncs/FILE.cpp** with list of functions in **FUNCS**.
* **call_cpp(FILE,FUNC)**: Call **FUNC** from C++ **FILE**.
* **delink_cpp(FILE)**: Delink C++ library.

### Example

In [4]:
model = BufferStockModelClass(solmethod='egm',Np=1500,Nm=1500,Na=1500)

for threads in [8,4,2,1]:
    for compiler in ['vs','intel']:
        
        print(f'compiler = {compiler}, threads = {threads},')
        model.par.cppthreads = threads
        tic,toc = model.solve_cpp(compiler=compiler)
        print(f'solved in {toc-tic:.1f} secs')
        model.checksum()
        print('')


compiler = vs, threads = 8,
cpp files loaded
solved in 3.2 secs
checksum: 3.401607450896838

compiler = intel, threads = 8,
cpp files loaded
solved in 2.6 secs
checksum: 3.401607450896838

compiler = vs, threads = 4,
cpp files loaded
solved in 6.4 secs
checksum: 3.401607450896838

compiler = intel, threads = 4,
cpp files loaded
solved in 5.1 secs
checksum: 3.401607450896838

compiler = vs, threads = 2,
cpp files loaded
solved in 12.7 secs
checksum: 3.401607450896838

compiler = intel, threads = 2,
cpp files loaded
solved in 9.9 secs
checksum: 3.401607450896838

compiler = vs, threads = 1,
cpp files loaded
solved in 25.2 secs
checksum: 3.401607450896838

compiler = intel, threads = 1,
cpp files loaded
solved in 19.5 secs
checksum: 3.401607450896838

